# Scenario 04: Deterministic Multi-Agent Workflows

**Estimated Time**: 45 minutes

## Learning Objectives
- Design deterministic workflows using Microsoft Agent Framework's `WorkflowBuilder`
- Implement sequential agent chains with `add_edge()`
- Execute parallel agent operations with `asyncio.gather()`
- Handle conditional routing with condition functions
- Use middleware patterns for error handling and retry logic

## Prerequisites
- Completed Scenario 01 (Simple Agent + MCP)
- Understanding of async/await patterns in Python
- Azure OpenAI access configured in `.env`

## Part 1: Understanding Deterministic Workflows

### What Are Deterministic Workflows?

Deterministic workflows define a **fixed execution path** for coordinating multiple agents:

- **Predictable**: Same inputs produce same execution order
- **Debuggable**: Clear step-by-step execution trace
- **Testable**: Each step can be tested independently
- **Recoverable**: Built-in error handling and retry logic

### Microsoft Agent Framework Approach

Instead of building custom workflow engines, we use the official `WorkflowBuilder` API:

| Custom Approach | Agent Framework |
|-----------------|-----------------|
| `WorkflowEngine.add_step()` | `WorkflowBuilder.add_edge()` |
| `SequentialStep` container | Chained `add_edge()` calls |
| `ParallelStep` container | `asyncio.gather()` with agents |
| `ConditionalStep` | `add_edge(condition=fn)` |
| Custom error strategies | Middleware patterns |

### When to Use Deterministic Workflows

- ETL pipelines with agent transformations
- Document processing (extract → analyze → summarize)
- Research workflows (search → analyze → report)
- Approval workflows with human-in-the-loop

## Part 2: Setting Up the Environment

In [17]:
# Load environment and configure paths
import sys
import os
from pathlib import Path

# Add project root to path
project_root = Path("..").resolve()
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

# Load environment variables (force override cached values)
from dotenv import load_dotenv
load_dotenv(project_root / ".env", override=True)

# IMPORTANT: Remove API key from env to force Entra ID auth
if "AZURE_OPENAI_API_KEY" in os.environ:
    del os.environ["AZURE_OPENAI_API_KEY"]
    print("⚠️ Removed AZURE_OPENAI_API_KEY from environment to force Entra ID auth")

# Verify Azure OpenAI configuration
assert os.getenv("AZURE_OPENAI_ENDPOINT"), "Missing AZURE_OPENAI_ENDPOINT"
assert os.getenv("AZURE_OPENAI_DEPLOYMENT"), "Missing AZURE_OPENAI_DEPLOYMENT"

print(f"✅ Project root: {project_root}")
print(f"✅ Azure OpenAI endpoint: {os.getenv('AZURE_OPENAI_ENDPOINT')}")

⚠️ Removed AZURE_OPENAI_API_KEY from environment to force Entra ID auth
✅ Project root: C:\Users\jonasrotter\OneDrive - Microsoft\Desktop\Jonas Privat\MyCodingProjects\agents-workshop
✅ Azure OpenAI endpoint: https://aistudiojonasr5312406741.openai.azure.com/


In [18]:
# Import Microsoft Agent Framework components
import os
import asyncio
from typing import Any

# Agent Framework imports
from agent_framework import WorkflowBuilder, ChatAgent
from agent_framework.azure import AzureOpenAIChatClient
from azure.identity import DefaultAzureCredential, get_bearer_token_provider

# Telemetry setup
from src.common.telemetry import setup_telemetry, get_tracer
setup_telemetry()
tracer = get_tracer(__name__)

# Create token provider for Azure OpenAI with correct scope
credential = DefaultAzureCredential()
token_provider = get_bearer_token_provider(
    credential, 
    "https://cognitiveservices.azure.com/.default"
)

# Create Azure OpenAI chat client with token provider
# Note: The parameter is 'ad_token_provider' not 'azure_ad_token_provider'
chat_client = AzureOpenAIChatClient(
    ad_token_provider=token_provider,
    endpoint=os.environ["AZURE_OPENAI_ENDPOINT"],
    deployment_name=os.environ["AZURE_OPENAI_DEPLOYMENT"],
    api_version=os.environ.get("AZURE_OPENAI_VERSION", "2024-12-01-preview"),
)

print("✅ Microsoft Agent Framework components imported!")
print(f"✅ Azure OpenAI client created with deployment: {os.environ['AZURE_OPENAI_DEPLOYMENT']}")

✅ Microsoft Agent Framework components imported!
✅ Azure OpenAI client created with deployment: gpt-4.1-mini


## Part 3: Basic Sequential Workflow

Let's create a simple sequential workflow using `WorkflowBuilder` and `ChatAgent`.

In [19]:
# Create specialized agents for each step
step1_agent = chat_client.create_agent(
    name="step1_agent",
    instructions="You are Step 1. Add the prefix '[Step 1]:' to the beginning of any message you receive. Return ONLY the prefixed message, nothing else."
)

step2_agent = chat_client.create_agent(
    name="step2_agent",
    instructions="You are Step 2. Add the prefix '[Step 2]:' to the beginning of any message you receive. Return ONLY the prefixed message, nothing else."
)

step3_agent = chat_client.create_agent(
    name="step3_agent",
    instructions="You are Step 3. Add the prefix '[Step 3]:' to the beginning of any message you receive. Return ONLY the prefixed message, nothing else."
)

# Build workflow with sequential edges
workflow = (
    WorkflowBuilder()
    .set_start_executor(step1_agent)
    .add_edge(step1_agent, step2_agent)
    .add_edge(step2_agent, step3_agent)
    .build()
)

# Execute the workflow
import time
start = time.time()
events = await workflow.run("Hello, Workflow!")
duration = (time.time() - start) * 1000

# Collect results
final_output = None
for event in events:
    # Only process events that have executor_id (agent events)
    if hasattr(event, 'executor_id') and hasattr(event, 'data'):
        executor_id = event.executor_id
        data = event.data
        # Handle AgentRunResponse objects
        if hasattr(data, 'text'):
            output_text = data.text
        elif hasattr(data, 'messages') and data.messages:
            # Get text from the last message
            last_msg = data.messages[-1]
            if hasattr(last_msg, 'content'):
                output_text = str(last_msg.content)
            else:
                output_text = str(last_msg)
        else:
            output_text = str(data)
        final_output = output_text
        truncated = output_text[:100] + "..." if len(output_text) > 100 else output_text
        print(f"[{executor_id}]: {truncated}")

print(f"\n✅ Workflow completed in {duration:.2f}ms")
print(f"Final output: {final_output}")

No outgoing edges found for executor step3_agent; dropping messages.


[step1_agent]: Hello, Workflow!
[step1_agent]: [Step 1]: Hello, Workflow! How can I assist you today?
[step1_agent]: [AgentExecutorResponse(executor_id='step1_agent', agent_run_response=<agent_framework._types.AgentRu...
[step2_agent]: AgentExecutorResponse(executor_id='step1_agent', agent_run_response=<agent_framework._types.AgentRun...
[step2_agent]: [Step 2]: Hello, Workflow! How can I assist you today?
[step2_agent]: [AgentExecutorResponse(executor_id='step2_agent', agent_run_response=<agent_framework._types.AgentRu...
[step3_agent]: AgentExecutorResponse(executor_id='step2_agent', agent_run_response=<agent_framework._types.AgentRun...
[step3_agent]: [Step 3]: Hello, Workflow!
[step3_agent]: [AgentExecutorResponse(executor_id='step3_agent', agent_run_response=<agent_framework._types.AgentRu...

✅ Workflow completed in 6017.08ms
Final output: [AgentExecutorResponse(executor_id='step3_agent', agent_run_response=<agent_framework._types.AgentRunResponse object at 0x000002CF5D532510>, f

In [20]:
# Stream workflow events in real-time
print("=== Streaming Workflow Execution ===\n")

async for event in workflow.run_stream("Hello, Streaming Workflow!"):
    if hasattr(event, 'executor_id') and hasattr(event, 'data'):
        data = event.data
        # Handle AgentRunResponseUpdate objects
        if hasattr(data, 'text'):
            output_text = data.text
        elif hasattr(data, 'delta') and data.delta:
            output_text = str(data.delta)
        else:
            output_text = str(data)[:80]
        print(f"📤 [{event.executor_id}]: {output_text}...")
    elif hasattr(event, 'type'):
        print(f"📍 Event: {event.type}")

=== Streaming Workflow Execution ===

📤 [step1_agent]: Hello, Streaming Workflow!...
📤 [step1_agent]: ...
📤 [step1_agent]: [...
📤 [step1_agent]: Step...
📤 [step1_agent]:  ...
📤 [step1_agent]: 1...
📤 [step1_agent]: ]:...
📤 [step1_agent]:  Hello...
📤 [step1_agent]: ,...
📤 [step1_agent]:  Streaming...
📤 [step1_agent]:  Workflow...
📤 [step1_agent]: !...
📤 [step1_agent]:  How...
📤 [step1_agent]:  can...
📤 [step1_agent]:  I...
📤 [step1_agent]:  assist...
📤 [step1_agent]:  you...
📤 [step1_agent]:  today...
📤 [step1_agent]: ?...
📤 [step1_agent]: ...
📤 [step1_agent]: ...
📤 [step1_agent]: [AgentExecutorResponse(executor_id='step1_agent', agent_run_response=<agent_fram...
📤 [step2_agent]: AgentExecutorResponse(executor_id='step1_agent', agent_run_response=<agent_frame...
📤 [step2_agent]: ...
📤 [step2_agent]: [...
📤 [step2_agent]: Step...
📤 [step2_agent]:  ...
📤 [step2_agent]: 2...
📤 [step2_agent]: ]:...
📤 [step2_agent]:  Hello...
📤 [step2_agent]: ,...
📤 [step2_agent]:  Streaming...
📤 [step2_agent

No outgoing edges found for executor step3_agent; dropping messages.


## Part 4: Chained Agent Pipelines

Create more complex pipelines by chaining multiple agents with `add_edge()`.

In [21]:
# Create a text processing pipeline with specialized agents

uppercase_agent = chat_client.create_agent(
    name="uppercase",
    instructions="Convert the input text to UPPERCASE. Return ONLY the uppercase text."
)

reverse_agent = chat_client.create_agent(
    name="reverse",
    instructions="Reverse the order of characters in the input text. Return ONLY the reversed text."
)

count_agent = chat_client.create_agent(
    name="count",
    instructions="Count the characters in the input and append ' (length: N)' to the text. Return the text with the count."
)

# Build text processing pipeline
text_pipeline = (
    WorkflowBuilder()
    .set_start_executor(uppercase_agent)
    .add_edge(uppercase_agent, reverse_agent)
    .add_edge(reverse_agent, count_agent)
    .build()
)

# Execute pipeline
input_text = "hello world"
print(f"Input: '{input_text}'")

events = await text_pipeline.run(input_text)

final_result = None
for event in events:
    if hasattr(event, 'data'):
        final_result = event.data

print(f"Output: '{final_result}'")
print("\nNote: Each agent in the chain processes the output of the previous agent.")

Input: 'hello world'


No outgoing edges found for executor count; dropping messages.


Output: 'None'

Note: Each agent in the chain processes the output of the previous agent.


## Part 5: Parallel Execution with asyncio.gather()

Execute multiple agents concurrently when their operations are independent.

In [22]:
import asyncio
import time

# Create parallel analysis agents
sentiment_agent = chat_client.create_agent(
    name="sentiment",
    instructions="Analyze the sentiment of the input text. Return a JSON object with: {\"sentiment\": \"positive|negative|neutral\", \"confidence\": 0.0-1.0}"
)

entities_agent = chat_client.create_agent(
    name="entities",
    instructions="Extract named entities from the input text. Return a JSON object with: {\"entities\": [{\"text\": \"...\", \"type\": \"PERSON|ORG|LOCATION\"}]}"
)

topics_agent = chat_client.create_agent(
    name="topics",
    instructions="Identify the main topics in the input text. Return a JSON object with: {\"topics\": [\"topic1\", \"topic2\", ...]}"
)

def get_response_text(response) -> str:
    """Extract text from AgentRunResponse."""
    if hasattr(response, 'text') and response.text:
        return response.text
    if hasattr(response, 'messages') and response.messages:
        last_msg = response.messages[-1]
        if hasattr(last_msg, 'content'):
            return str(last_msg.content)
    return str(response)

async def parallel_analysis(text: str) -> dict:
    """Execute multiple agents in parallel using asyncio.gather()."""
    start = time.time()
    
    # Run all agents concurrently
    results = await asyncio.gather(
        sentiment_agent.run(text),
        entities_agent.run(text),
        topics_agent.run(text),
    )
    
    duration = (time.time() - start) * 1000
    
    return {
        "sentiment": get_response_text(results[0]),
        "entities": get_response_text(results[1]),
        "topics": get_response_text(results[2]),
        "duration_ms": duration,
    }

# Test parallel execution
sample_text = "Apple CEO Tim Cook announced the new iPhone at the company's headquarters in Cupertino, California."

result = await parallel_analysis(sample_text)

print(f"⏱️ Duration: {result['duration_ms']:.2f}ms (parallel execution!)")
print(f"\n📊 Sentiment: {result['sentiment'][:100]}...")
print(f"\n🏷️ Entities: {result['entities'][:100]}...")
print(f"\n📑 Topics: {result['topics'][:100]}...")

⏱️ Duration: 1703.67ms (parallel execution!)

📊 Sentiment: {"sentiment": "neutral", "confidence": 0.85}...

🏷️ Entities: {
  "entities": [
    {"text": "Apple", "type": "ORG"},
    {"text": "Tim Cook", "type": "PERSON"},
...

📑 Topics: {"topics": ["Apple", "Tim Cook", "iPhone announcement", "Cupertino", "Apple headquarters"]}...


## Part 6: Conditional Branching with add_edge(condition=...)

Route to different agents based on runtime conditions.

In [23]:
# Create router and specialized agents
router_agent = chat_client.create_agent(
    name="router",
    instructions="Determine if the input text is long (>100 chars) or short. Return exactly 'LONG' or 'SHORT'."
)

summarize_agent = chat_client.create_agent(
    name="summarize",
    instructions="Summarize the long text into 2-3 sentences. Be concise."
)

expand_agent = chat_client.create_agent(
    name="expand",
    instructions="Expand the short text into a more detailed explanation (3-4 sentences)."
)

def extract_text_from_response(data) -> str:
    """Extract text from AgentExecutorResponse or string."""
    if isinstance(data, str):
        return data
    if hasattr(data, 'agent_run_response'):
        response = data.agent_run_response
        if hasattr(response, 'text') and response.text:
            return response.text
        if hasattr(response, 'messages') and response.messages:
            last_msg = response.messages[-1]
            if hasattr(last_msg, 'content'):
                return str(last_msg.content)
    return str(data)

# Define condition functions that work with AgentExecutorResponse
def is_long_text(data) -> bool:
    """Check if router response indicates LONG."""
    text = extract_text_from_response(data)
    return "LONG" in text.upper()

def is_short_text(data) -> bool:
    """Check if router response indicates SHORT."""
    text = extract_text_from_response(data)
    return "SHORT" in text.upper()

# Build conditional workflow
conditional_workflow = (
    WorkflowBuilder()
    .set_start_executor(router_agent)
    .add_edge(router_agent, summarize_agent, condition=is_long_text)
    .add_edge(router_agent, expand_agent, condition=is_short_text)
    .build()
)

# Test with short text
short_text = "AI is transforming industries."
print(f"Short text ({len(short_text)} chars): '{short_text}'")
events = await conditional_workflow.run(short_text)
for event in events:
    if hasattr(event, 'executor_id') and hasattr(event, 'data'):
        text = extract_text_from_response(event.data)
        print(f"[{event.executor_id}]: {text[:150]}...")

print()

# Test with long text
long_text = "Artificial intelligence has been making remarkable strides in recent years, with applications spanning healthcare, finance, transportation, and entertainment. Machine learning algorithms are now capable of diagnosing diseases, predicting market trends, and driving autonomous vehicles."
print(f"Long text ({len(long_text)} chars): '{long_text[:50]}...'")
events = await conditional_workflow.run(long_text)
for event in events:
    if hasattr(event, 'executor_id') and hasattr(event, 'data'):
        text = extract_text_from_response(event.data)
        print(f"[{event.executor_id}]: {text[:150]}...")

Short text (30 chars): 'AI is transforming industries.'


No outgoing edges found for executor expand; dropping messages.


[router]: AI is transforming industries....
[router]: SHORT...
[router]: [AgentExecutorResponse(executor_id='router', agent_run_response=<agent_framework._types.AgentRunResponse object at 0x000002CF5CD1E9F0>, full_conversat...
[expand]: SHORT...
[expand]: Artificial intelligence (AI) is revolutionizing various industries by automating tasks, enhancing decision-making, and improving efficiency. In sector...
[expand]: [AgentExecutorResponse(executor_id='expand', agent_run_response=<agent_framework._types.AgentRunResponse object at 0x000002CF5D4FDC40>, full_conversat...

Long text (285 chars): 'Artificial intelligence has been making remarkable...'


No outgoing edges found for executor summarize; dropping messages.


[router]: Artificial intelligence has been making remarkable strides in recent years, with applications spanning healthcare, finance, transportation, and entert...
[router]: LONG...
[router]: [AgentExecutorResponse(executor_id='router', agent_run_response=<agent_framework._types.AgentRunResponse object at 0x000002CF5E5B7020>, full_conversat...
[summarize]: LONG...
[summarize]: Artificial intelligence has advanced significantly, impacting various fields such as healthcare, finance, transportation, and entertainment. Machine l...
[summarize]: [AgentExecutorResponse(executor_id='summarize', agent_run_response=<agent_framework._types.AgentRunResponse object at 0x000002CF5D532480>, full_conver...


## Part 7: Data Transformation Between Agents

Pass structured data between agents using prompt templates.

In [24]:
# Data transformation through prompt templates

keyword_extractor = chat_client.create_agent(
    name="keyword_extractor",
    instructions="""Extract the top 5 keywords from the input text.
Return ONLY a comma-separated list of keywords, nothing else.
Example: keyword1, keyword2, keyword3, keyword4, keyword5"""
)

def get_text_from_run_response(response) -> str:
    """Extract text from AgentRunResponse."""
    if hasattr(response, 'text') and response.text:
        return response.text
    if hasattr(response, 'messages') and response.messages:
        last_msg = response.messages[-1]
        if hasattr(last_msg, 'content'):
            return str(last_msg.content)
    return str(response)

async def extract_and_transform(text: str) -> dict:
    """Extract keywords and return structured data."""
    # Run keyword extraction
    result = await keyword_extractor.run(text)
    
    # Transform response to structured format
    result_text = get_text_from_run_response(result)
    keywords = [k.strip() for k in result_text.split(',')]
    
    return {
        "original_text": text[:50] + "...",
        "keywords": keywords[:5],
        "keyword_count": len(keywords),
    }

# Test data transformation
sample = "The artificial intelligence revolution is transforming industries worldwide through machine learning and deep neural networks."

result = await extract_and_transform(sample)

print(f"Original: {result['original_text']}")
print(f"Keywords: {result['keywords']}")
print(f"Count: {result['keyword_count']}")

Original: The artificial intelligence revolution is transfor...
Keywords: ['artificial intelligence', 'machine learning', 'deep neural networks', 'revolution', 'industries']
Count: 5


## Part 8: Error Handling with Middleware

Use middleware patterns for retry logic and error handling.

In [25]:
# Define a retry middleware for agent calls
import asyncio
from typing import Callable, Any

class RetryMiddleware:
    """Middleware that retries failed agent calls with exponential backoff."""
    
    def __init__(self, max_retries: int = 3, base_delay: float = 1.0, max_delay: float = 10.0):
        self.max_retries = max_retries
        self.base_delay = base_delay
        self.max_delay = max_delay
    
    async def execute(self, func: Callable, *args, **kwargs) -> Any:
        """Execute function with retry logic."""
        delay = self.base_delay
        last_error = None
        
        for attempt in range(self.max_retries + 1):
            try:
                return await func(*args, **kwargs)
            except Exception as e:
                last_error = e
                if attempt < self.max_retries:
                    print(f"⚠️ Attempt {attempt + 1} failed: {e}")
                    print(f"   Retrying in {delay:.1f}s...")
                    await asyncio.sleep(delay)
                    delay = min(delay * 2, self.max_delay)
        
        raise last_error

# Example: Safe workflow wrapper with error handling
async def safe_agent_call(agent, prompt: str, fallback: str = "Unable to process request."):
    """Execute agent with fallback on failure."""
    try:
        return await agent.run(prompt)
    except Exception as e:
        print(f"❌ Agent call failed: {e}")
        return fallback

print("✅ Middleware patterns defined")

✅ Middleware patterns defined


In [26]:
# Demonstrate safe workflow execution with error handling

async def robust_workflow(text: str) -> dict:
    """Execute a workflow with comprehensive error handling."""
    results = {}
    
    # Step 1: Extract keywords with fallback
    keyword_result = await safe_agent_call(
        keyword_extractor,
        text,
        fallback="keyword extraction failed"
    )
    results["keywords"] = get_text_from_run_response(keyword_result) if hasattr(keyword_result, 'messages') else keyword_result
    
    # Step 2: Analyze sentiment with fallback
    sentiment_result = await safe_agent_call(
        sentiment_agent,
        text,
        fallback='{"sentiment": "unknown", "confidence": 0.0}'
    )
    results["sentiment"] = get_text_from_run_response(sentiment_result) if hasattr(sentiment_result, 'messages') else sentiment_result
    
    return results

# Test robust workflow
result = await robust_workflow("The product launch was a huge success!")
print(f"Keywords: {result['keywords'][:80]}...")
print(f"Sentiment: {result['sentiment'][:80]}...")

Keywords: product, launch, huge, success,...
Sentiment: {"sentiment": "positive", "confidence": 0.95}...


In [27]:
# Demonstrate circuit breaker pattern for resilience

class CircuitBreaker:
    """Simple circuit breaker to prevent cascading failures."""
    
    def __init__(self, failure_threshold: int = 3, reset_timeout: float = 30.0):
        self.failure_threshold = failure_threshold
        self.reset_timeout = reset_timeout
        self.failures = 0
        self.last_failure_time = None
        self.state = "CLOSED"  # CLOSED, OPEN, HALF_OPEN
    
    def can_execute(self) -> bool:
        """Check if execution is allowed."""
        if self.state == "CLOSED":
            return True
        
        if self.state == "OPEN":
            import time
            if time.time() - self.last_failure_time > self.reset_timeout:
                self.state = "HALF_OPEN"
                return True
            return False
        
        return True  # HALF_OPEN allows one attempt
    
    def record_success(self):
        """Record successful execution."""
        self.failures = 0
        self.state = "CLOSED"
    
    def record_failure(self):
        """Record failed execution."""
        import time
        self.failures += 1
        self.last_failure_time = time.time()
        
        if self.failures >= self.failure_threshold:
            self.state = "OPEN"
            print(f"🔴 Circuit breaker OPEN (failures: {self.failures})")

circuit_breaker = CircuitBreaker()
print(f"✅ Circuit breaker initialized (state: {circuit_breaker.state})")

✅ Circuit breaker initialized (state: CLOSED)


In [28]:
# Demonstrate timeout handling for agent calls

def extract_text(response) -> str:
    """Extract text from AgentRunResponse or return string."""
    if isinstance(response, str):
        return response
    if hasattr(response, 'text') and response.text:
        return response.text
    if hasattr(response, 'messages') and response.messages:
        last_msg = response.messages[-1]
        if hasattr(last_msg, 'content'):
            return str(last_msg.content)
    return str(response)

async def agent_with_timeout(agent, prompt: str, timeout_seconds: float = 30.0):
    """Execute agent with timeout."""
    try:
        result = await asyncio.wait_for(
            agent.run(prompt),
            timeout=timeout_seconds
        )
        return {"status": "success", "result": extract_text(result)}
    except asyncio.TimeoutError:
        return {"status": "timeout", "result": f"Agent timed out after {timeout_seconds}s"}
    except Exception as e:
        return {"status": "error", "result": str(e)}

# Test timeout handling
result = await agent_with_timeout(
    sentiment_agent,
    "This is a great day!",
    timeout_seconds=30.0
)

print(f"Status: {result['status']}")
print(f"Result: {result['result'][:100]}...")

Status: success
Result: {"sentiment": "positive", "confidence": 0.95}...


## Part 9: Retry Strategies with Middleware

The Agent Framework uses middleware patterns for cross-cutting concerns like retries. We've implemented a `RetryMiddleware` that can be composed with any agent call.

Key benefits:
- **Composable**: Wrap any agent call with retry behavior
- **Configurable**: Customize retry counts, delays, and conditions
- **Transparent**: The underlying agent code doesn't need to change

In [29]:
# Simulate a flaky operation and test retry middleware

import random

class FlakyAgent:
    """Agent that randomly fails to demonstrate retry patterns."""
    
    def __init__(self, failure_rate: float = 0.7):
        self.failure_rate = failure_rate
        self.call_count = 0
    
    async def run(self, prompt: str) -> str:
        self.call_count += 1
        if random.random() < self.failure_rate:
            raise Exception(f"Simulated failure on call #{self.call_count}")
        return f"Success on call #{self.call_count}: Processed '{prompt[:30]}...'"

# Create flaky agent and retry middleware
flaky_agent = FlakyAgent(failure_rate=0.7)
retry_middleware = RetryMiddleware(max_retries=5, base_delay=0.1, max_delay=1.0)

# Execute with retry
async def run_with_retry():
    return await retry_middleware.execute(
        flaky_agent.run,
        "Please analyze this important data"
    )

try:
    result = await run_with_retry()
    print(f"✓ {result}")
except Exception as e:
    print(f"✗ Failed after all retries: {e}")

print(f"Total API calls: {flaky_agent.call_count}")

⚠️ Attempt 1 failed: Simulated failure on call #1
   Retrying in 0.1s...
⚠️ Attempt 2 failed: Simulated failure on call #2
   Retrying in 0.2s...
⚠️ Attempt 3 failed: Simulated failure on call #3
   Retrying in 0.4s...
⚠️ Attempt 4 failed: Simulated failure on call #4
   Retrying in 0.8s...
⚠️ Attempt 5 failed: Simulated failure on call #5
   Retrying in 1.0s...
✓ Success on call #6: Processed 'Please analyze this important ...'
Total API calls: 6


## Part 10: The Framework's WorkflowBuilder Pattern

The Microsoft Agent Framework provides a `WorkflowBuilder` class that enables declarative workflow construction. Here's the complete pattern for building complex workflows:

```python
# The canonical pattern for Agent Framework workflows:
workflow = (
    WorkflowBuilder()
    .set_start_executor(agent_1)
    .add_edge(agent_1, agent_2)
    .add_edge(agent_2, agent_3, condition=some_condition)
    .build()
)
```

**Key Components**:
- `set_start_executor()`: Define the entry point
- `add_edge()`: Create connections between agents
- `condition=`: Add conditional routing
- `build()`: Finalize the workflow

In [30]:
# Build a complete analysis workflow using WorkflowBuilder

# Create specialized agents for different analysis tasks
extract_agent = chat_client.create_agent(
    name="Extractor",
    instructions="You are a data extraction specialist. Extract key facts from text."
)

analyze_agent = chat_client.create_agent(
    name="Analyzer", 
    instructions="You are an analytical AI. Analyze patterns and draw conclusions."
)

format_agent = chat_client.create_agent(
    name="Formatter",
    instructions="You are a formatting specialist. Format analysis results professionally."
)

# Build workflow with chained agents
analysis_workflow = (
    WorkflowBuilder()
    .set_start_executor(extract_agent)
    .add_edge(extract_agent, analyze_agent)
    .add_edge(analyze_agent, format_agent)
    .build()
)

# Execute the workflow
sample_text = """
Q3 2024 Results: Revenue increased 15% YoY to $2.4B.
Operating margin improved to 28%. Cloud services grew 45%.
New customer acquisition up 22% with 95% retention rate.
"""

print("=== Complete Analysis Workflow ===\n")
print(f"Input Text:\n{sample_text}\n")

# Run extraction
extract_result = await extract_agent.run(f"Extract key metrics from: {sample_text}")
print(f"1. Extraction:\n{extract_result}\n")

# Run analysis
analyze_result = await analyze_agent.run(f"Analyze these extracted facts: {extract_result}")
print(f"2. Analysis:\n{analyze_result}\n")

# Run formatting
format_result = await format_agent.run(f"Format this analysis professionally: {analyze_result}")
print(f"3. Final Output:\n{format_result}")

=== Complete Analysis Workflow ===

Input Text:

Q3 2024 Results: Revenue increased 15% YoY to $2.4B.
Operating margin improved to 28%. Cloud services grew 45%.
New customer acquisition up 22% with 95% retention rate.


1. Extraction:
- Quarter: Q3 2024  
- Revenue: $2.4 billion (15% year-over-year increase)  
- Operating margin: 28%  
- Cloud services growth: 45%  
- New customer acquisition: 22% increase  
- Customer retention rate: 95%

2. Analysis:
Based on the extracted facts about the company's Q3 2024 performance, several insights and patterns emerge:

1. **Strong Overall Revenue Growth**  
   - The company reported $2.4 billion in revenue, a 15% increase compared to the same quarter last year.  
   - This indicates solid market demand and effective sales strategies.

2. **High Operating Margin Indicates Efficient Operations**  
   - An operating margin of 28% is relatively high, suggesting the company manages costs well and maintains strong profitability.  
   - This margin sup

## Part 11: Workflow Validation and Testing

When building deterministic workflows, validation is critical:

1. **Pre-execution validation**: Check all agents are properly configured
2. **Runtime monitoring**: Track execution through each step
3. **Post-execution validation**: Verify outputs meet expectations

The Agent Framework's approach makes testing easier because each component is independent.

In [31]:
# Workflow validation utilities

def validate_agent(agent) -> dict:
    """Validate agent configuration before workflow execution."""
    issues = []
    agent_name = getattr(agent, 'name', 'Unknown')
    
    # Check required attributes
    if not agent_name or agent_name == 'Unknown':
        issues.append("Agent missing name")
    
    # Check if agent has run method
    if not hasattr(agent, 'run') or not callable(getattr(agent, 'run', None)):
        issues.append("Agent missing run() method")
    
    return {
        "agent": agent_name,
        "valid": len(issues) == 0,
        "issues": issues
    }

def validate_workflow_agents(*agents) -> dict:
    """Validate all agents in a workflow."""
    results = [validate_agent(a) for a in agents]
    all_valid = all(r["valid"] for r in results)
    
    return {
        "valid": all_valid,
        "agent_results": results,
        "summary": f"{sum(1 for r in results if r['valid'])}/{len(results)} agents valid"
    }

# Validate our workflow agents
validation = validate_workflow_agents(
    sentiment_agent,
    entities_agent,
    topics_agent,
    extract_agent,
    analyze_agent,
    format_agent
)

print("=== Workflow Validation ===\n")
print(f"Overall: {'✓ PASS' if validation['valid'] else '✗ FAIL'}")
print(f"Summary: {validation['summary']}\n")

for result in validation['agent_results']:
    status = "✓" if result['valid'] else "✗"
    print(f"  {status} {result['agent']}: {'OK' if result['valid'] else result['issues']}")

=== Workflow Validation ===

Overall: ✓ PASS
Summary: 6/6 agents valid

  ✓ sentiment: OK
  ✓ entities: OK
  ✓ topics: OK
  ✓ Extractor: OK
  ✓ Analyzer: OK
  ✓ Formatter: OK


## Part 12: Hands-on Exercise

**Challenge**: Build a content moderation workflow using the patterns learned.

Your workflow should:
1. **Classify** content (safe/unsafe) using a classification agent
2. **Route** based on classification (conditional logic)
3. **Process** safe content through enhancement
4. **Flag** unsafe content for review

Use the Microsoft Agent Framework patterns:
- `ChatAgent` for each processing step
- `asyncio.gather()` if parallel processing needed
- Conditional routing with if/else based on agent output

In [32]:
# Exercise: Build a content moderation workflow

def get_response_text(response) -> str:
    """Extract text from AgentRunResponse."""
    if isinstance(response, str):
        return response
    if hasattr(response, 'text') and response.text:
        return response.text
    if hasattr(response, 'messages') and response.messages:
        last_msg = response.messages[-1]
        if hasattr(last_msg, 'content'):
            return str(last_msg.content)
    return str(response)

# Step 1: Create the agents
classifier_agent = chat_client.create_agent(
    name="Classifier",
    instructions="""You are a content classifier. 
    Analyze text and respond with exactly one word: SAFE or UNSAFE.
    SAFE = appropriate professional content
    UNSAFE = inappropriate, harmful, or policy-violating content"""
)

enhancer_agent = chat_client.create_agent(
    name="Enhancer",
    instructions="You are a content enhancer. Improve the quality and clarity of safe content."
)

flagger_agent = chat_client.create_agent(
    name="Flagger",
    instructions="You are a content reviewer. Document why content was flagged and suggest remediation."
)

# Step 2: Build the moderation workflow
async def moderate_content(content: str) -> dict:
    """Content moderation workflow using Agent Framework patterns."""
    
    # Classify content
    classification_response = await classifier_agent.run(f"Classify this content: {content}")
    classification_text = get_response_text(classification_response)
    is_safe = "SAFE" in classification_text.upper()
    
    # Route based on classification
    if is_safe:
        # Process safe content
        enhanced = await enhancer_agent.run(f"Enhance this content: {content}")
        return {
            "status": "approved",
            "classification": "SAFE",
            "result": get_response_text(enhanced)
        }
    else:
        # Flag unsafe content
        flag_report = await flagger_agent.run(f"Review and document why this is flagged: {content}")
        return {
            "status": "flagged",
            "classification": "UNSAFE",
            "result": get_response_text(flag_report)
        }

# Test the workflow
test_contents = [
    "Our Q3 earnings exceeded expectations with 15% growth.",
    "Meeting scheduled for Tuesday to discuss project timeline."
]

print("=== Content Moderation Workflow ===\n")
for content in test_contents:
    print(f"Input: {content[:50]}...")
    result = await moderate_content(content)
    print(f"Status: {result['status'].upper()}")
    print(f"Classification: {result['classification']}")
    print(f"Output: {result['result'][:100]}...\n")

=== Content Moderation Workflow ===

Input: Our Q3 earnings exceeded expectations with 15% gro...
Status: APPROVED
Classification: SAFE
Output: Our Q3 earnings surpassed expectations, achieving an impressive 15% growth....

Input: Meeting scheduled for Tuesday to discuss project t...
Status: APPROVED
Classification: SAFE
Output: A meeting is scheduled for Tuesday to discuss and finalize the project timeline....



## Summary

In this notebook, we explored deterministic workflow orchestration using the **Microsoft Agent Framework**:

### Key Patterns Learned:

| Custom Code | Agent Framework Equivalent |
|-------------|---------------------------|
| `WorkflowEngine.add_step()` | `WorkflowBuilder().add_edge()` |
| `SequentialStep([steps])` | Chained `add_edge()` calls |
| `ParallelStep([steps])` | `asyncio.gather()` with agents |
| `ConditionalStep(cond, a, b)` | `add_edge(condition=fn)` |
| `RetryConfig` | `RetryMiddleware` pattern |
| `ErrorStrategy` enum | `CircuitBreaker` pattern |

### Framework Benefits:
- **Less custom code**: Leverage battle-tested framework components
- **Standard patterns**: Use industry-standard async patterns
- **Composability**: Combine agents and middleware flexibly
- **Testability**: Each component is independently testable

### Next Steps:
- **Notebook 05**: Declarative agent configuration with YAML
- **Notebook 06**: Multi-agent discussions and collaboration
- **Notebook 07**: Evaluation and prompt evolution